In [33]:
import pandas as pd
pd.options.display.max_columns = 99

In [34]:
customer = pd.read_csv('./data/Customer.csv')
isin = pd.read_csv('./data/Isin.csv')
market = pd.read_csv('./data/Market.csv')
marketMacro = pd.read_csv('./data/MarketData_Macro.csv')
trade = pd.read_csv('./data/Trade.csv')
challenge = pd.read_csv('./data/Challenge_20180423.csv')
All = [customer, isin, market, marketMacro, trade, challenge]

In [35]:
All = [customer, isin, market, trade]

In [36]:
for item in All:
    print (item.columns)
    print (item.shape)
    print ("\n")

Index(['CustomerIdx', 'Sector', 'Subsector', 'Region', 'Country'], dtype='object')
(3471, 5)


Index(['IsinIdx', 'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey',
       'Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'IssuedAmount', 'CouponType'],
      dtype='object')
(27411, 17)


Index(['IsinIdx', 'DateKey', 'Price', 'Yield', 'ZSpread'], dtype='object')
(9867747, 5)


Index(['TradeDateKey', 'CustomerIdx', 'IsinIdx', 'BuySell', 'NotionalEUR',
       'Price', 'TradeStatus', 'CustomerInterest'],
      dtype='object')
(6762021, 8)




In [37]:
trade.head()

,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
0,20161207,2789,8478,Sell,653168.0,0.0,Unknown,1.0
1,20170329,2574,14562,Buy,1656487.0,0.0,Unknown,1.0
2,20170418,2574,4747,Buy,939673.0,0.0,Unknown,1.0
3,20170310,2574,9885,Sell,708082.0,0.0,Unknown,1.0
4,20161116,2574,8885,Buy,1147709.0,0.0,Unknown,1.0


In [38]:
#1
one = trade.join(isin.set_index('IsinIdx'), how='left')
len(one)

6762021

In [39]:
#2
two = one.join(customer.set_index('CustomerIdx'), how='left', lsuffix='_two', rsuffix='_customer')
len(two)

6762021

In [40]:
two.head()

,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region_two,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType,Sector,Subsector,Region_customer,Country
0,20161207,2789,8478,Sell,653168.0,0.0,Unknown,1.0,238.0,20381231.0,20051129.0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN,Asset Managers & Hedge Funds,Independent Asset Manager,Asia Pacific,HONG KONG
1,20170329,2574,14562,Buy,1656487.0,0.0,Unknown,1.0,238.0,20331231.0,20051129.0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED,Asset Managers & Hedge Funds,Asset Mgr owned by Bank/Insur.,Americas,BRAZIL
2,20170418,2574,4747,Buy,939673.0,0.0,Unknown,1.0,238.0,20331231.0,20051129.0,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.501245e+10,FIXED,Asset Managers & Hedge Funds,Hedge Fund,Americas,BRAZIL
3,20170310,2574,9885,Sell,708082.0,0.0,Unknown,1.0,236.0,20170417.0,20070417.0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,B,Government,Sovereign,Domestic,7.340076e+09,FIXED,Asset Managers & Hedge Funds,NaN,"Europe, Middle East and Africa",GERMANY
4,20161116,2574,8885,Buy,1147709.0,0.0,Unknown,1.0,234.0,20221004.0,20100222.0,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,3.058452e+09,FLOATING,Asset Owners,Insurance,"Europe, Middle East and Africa",GERMANY


In [42]:
two.to_csv("./data/Aggregated.csv",index=False)

In [43]:
from datetime import datetime, timedelta

In [44]:
trade['date'] = trade['TradeDateKey'].apply(lambda x: str(x)).apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:]).apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [45]:
trade['week'] = trade['date'].apply(lambda x: x - timedelta(days=x.weekday()))

In [52]:
grouped = trade.groupby(by=['week','CustomerIdx','IsinIdx','BuySell','TradeStatus'])['CustomerInterest'].mean().reset_index()

In [48]:
len(grouped)

6419052

In [53]:
#1
oneW = grouped.join(isin.set_index('IsinIdx'), how='left')
len(oneW)

6419052

In [54]:
twoW = oneW.join(customer.set_index('CustomerIdx'), how='left', lsuffix='_two', rsuffix='_customer')
len(twoW)

6419052

In [55]:
twoW.to_csv("./data/Aggregated_week.csv",index=False)